# Menganalisis risiko peminjam gagal membayar

Proyek Anda ialah menyiapkan laporan untuk bank bagian kredit. Anda harus mencari tahu pengaruh status perkawinan seorang nasabah dan jumlah anak terhadap probabilitas ketepatan waktu dalam melunasi pinjaman. Bank sudah memiliki beberapa data mengenai kelayakan kredit nasabah.

Laporan Anda akan dipertimbangkan pada saat membuat **penilaian kredit** untuk calon nasabah. **Penilaian kredit** digunakan untuk mengevaluasi kemampuan calon peminjam untuk melunasi pinjaman mereka.



In [2]:
# Memuat semua perpustakaan
import pandas as pd
import numpy as np

In [4]:
# muat data
try:
    data = pd.read_csv('data/credit_scoring_eng.csv')
except:
    data = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Soal 1. Eksplorasi Data

**Deskripsi Data**
- *`children`* - jumlah anak dalam keluarga
- *`days_employed`* - pengalaman kerja dalam hari
- *`dob_years`* - usia klien dalam tahun
- *`education`* - pendidikan klien
- *`education_id`* - tanda pengenal pendidikan
- *`family_status`* - status perkawinan
- *`family_status_id`* - tanda pengenal status perkawinan
- *`gender`* - jenis kelamin klien
- *`income_type`* - jenis pekerjaan
- *`debt`* - apakah klien memiliki hutang pembayaran pinjaman
- *`total_income`* - pendapatan bulanan
- *`purpose`* - tujuan mendapatkan pinjaman

In [5]:
# Mari kita lihat berapa banyak baris dan kolom yang dimiliki oleh dataset kita
data.shape

(21525, 12)

<div class="alert alert-block alert-info">
<b> Note</b> <a class="tocSkip"></a>
    
Terdapat 21,525 baris dan 12 Kolom
</div>


In [7]:
# mari menampilkan 10 baris pertama
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a><br/>
    

Berdasarkan hasil pengamatan:<br/>
- Terdapat data yang hilang <br/>
- Value education yang terlihat mirip namun berbeda penulisan besar dan kecilnya.<br/>
- Terdapat nilai negative pada value days_employed atau durasi lama bekerja.
</div>

In [8]:
# Mendapatkan informasi data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div class="alert alert-block alert-info">
<b>Note</b> <a class="tocSkip"></a>

Terdapat 2 kolom yang terdapat nilai yang hilang yaitu kolom days_employed dan total_income dengan jumlah yang sama
</div>

In [9]:
# Mari kita melihat tabel yang telah difilter dengan nilai yang hilang di kolom pertama dengan data yang hilang
data.loc[data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

Berdasarkan hasil pengamatan, <br/>
setiap index value days_employed yang hilang terdapat value total income yang hilang.
</div>

In [10]:
# Mari kita menerapkan beberapa persyaratan untuk memfilter data dan melihat jumlah baris dalam tabel yang difilter.
data_null = data.loc[(data['days_employed'].isna()) & (data['total_income'].isna())]
print(data_null.shape)

(2174, 12)


<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

Dapat dipastikan kedua kolom yang memiliki data yang hilang berada di baris yang sama.
</div>

In [11]:
print(len(data_null) / len(data)*100)

10.099883855981417


<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

dataset nilai yang hilang sebesar 10.09%
</div>

**Kesimpulan menengah**


<div class="alert alert-block alert-info">
<b>Note</b> <a class="tocSkip"></a>

dataset yang hilang terdapat di baris yang sama, sehingga selanjutnya akan dilakukan exploring data seperti mencari pola dan lakukan pengelompokan data untuk manipulasi data yang hilang bila diperlukan.
</div>

In [12]:
# Mari kita memeriksa klien yang tidak memiliki data tentang karakteristik yang teridentifikasi dan kolom dengan nilai yang hilang
data_clean = data.loc[~(data['days_employed'].isna()) & ~(data['total_income'].isna())]
data_clean.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000
mean,0.537388,63046.497661,43.255336,0.819079,0.972249,0.081184,26787.568355
std,1.371408,140827.311974,12.579170,0.550104,1.420596,0.273125,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [13]:
# Memeriksa distribusi
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


<div class="alert alert-block alert-info">
<b>Note</b> <a class="tocSkip"></a>

Sejauh ini tidak ditemukan perbedaan yang signifikan antara perbandingan data 'dengan dan tanpa' nilai yang hilang
    
Dugaan sementara, 
- Nilai yang hilang terjadi karena beberapa pelanggan yang enggan konfirmasi lama bekerja dan income karena harus menyertakan bukti lampiran dari tempat kerja terkait dan setelah mendapatkan bukti tersebut pelanggan lakukan input ulang.
- proses input nilai yang hilang dihold dan menunggu konfirmasi selanjutnya dari nasabah.
penanggulangan data tersebut dapat dilakukan pengecekan data duplikat dan melakukan pengkategorian untuk hasil yang lebih akurat.
    
Namun sebelum penanggulanggan dilakukan, kita akan melakukan exploring pattern dengan dataset lainnya.
</div>

In [14]:
# Periksa penyebab dan pola lain yang dapat mengakibatkan nilai yang hilang
print(data_null['debt'].value_counts())

debt
0    2004
1     170
Name: count, dtype: int64


**Kesimpulan menengah**

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>
    
Tidak terdapat pola terhadap riwayat nasabah pernah gagal melunasi pinjaman atau tidak.
</div>

In [15]:
# Memeriksa pola lainnya - jelaskan pola tersebut
pattern_exploring = ['family_status_id', 'education_id', 'gender', 'children']
for column in pattern_exploring:
    print(column)
    print(data_null[column].value_counts())
    print()

family_status_id
family_status_id
0    1237
1     442
4     288
3     112
2      95
Name: count, dtype: int64

education_id
education_id
1    1540
0     544
2      69
3      21
Name: count, dtype: int64

gender
gender
F    1484
M     690
Name: count, dtype: int64

children
children
 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: count, dtype: int64



**Kesimpulan**

<div class="alert alert-block alert-info">
<b>Note</b> <a class="tocSkip"></a>

Tidak ditemukan pola pada kolom lainnya.

Karena tidak ditemukannya pola pada data format data yang hilang. sehingga selanjutnya penanggulangan data yang hilang akan dilakukan yaitu:
- Melakukan pengecekan duplikat data
- Melakukan pengkategorian data
    
</div>

## Transformasi data

In [16]:
# Mari kita lihat semua nilai di kolom pendidikan untuk memeriksa ejaan apa yang perlu diperbaiki
print(data['education'].value_counts())

education
secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: count, dtype: int64


In [17]:
# Perbaiki register
data['education'] = data['education'].str.lower()

In [18]:
# Memeriksa semua nilai di kolom untuk memastikan kita telah memperbaikinya
print(data['education'].value_counts())

education
secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: count, dtype: int64


In [19]:
# Mari kita lihat distribusi nilai pada kolom `children`
print(data['children'].describe())
print()
print(data['children'].value_counts())

count    21525.000000
mean         0.538908
std          1.381587
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: count, dtype: int64


<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

Terdapat hal aneh pada kolom 'children' yaitu pada Value -1 dan 20. kemungkinan besar terdapat typo penulisan value tersebut. dan solusinya akan diperbaiki dengan nilai yang mendekati yaitu :
- Value -1 menjadi 1
- Value 20 menjadi 2
    
</div>

In [20]:
# [perbaiki data berdasarkan keputusan Anda]
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2

In [21]:
# Periksa kembali kolom `children` untuk memastikan semua telah diperbaiki
data['children'].value_counts()

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: count, dtype: int64

In [22]:
# Temukan data yang bermasalah di `days_employed`, jika terdapat masalah, dan hitung persentasenya
print(len(data[data['days_employed'] < 0]) / len(data))

0.7389547038327526


<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

Persentase nilai negatif pada data days_employed mencapai 73.89% yang merupakan nilai yang tinggi
    
</div>

In [23]:
print (len(data[data['days_employed'] > 36500]) / len(data))

0.16004645760743322


<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

Persentase nilai yang melebihi 36500 Days atau 100 Tahun pada data days_employed mencapai 16% dari data
    
</div>

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

Penanggulangan atas data tersebut yaitu :
1. Untuk nilai negatif akan dijadikan nilai yang positif.
2. Untuk nilai yang kurang masuk akal seperti lama bekerja lebih dari 100 Tahun akan dilakukan replace dengan nilai yang lebih masuk akal dengan metode:
- Melakukan convert hari menjadi tahun pada kolom baru dengan nama kolom ['experience']
- Melakukan perbandingan antara usia nasabah ['dob_years'] dengan ['experience'] dengan metode usia nasabah dikurangi dengan lama bekerja sehingga mendapatkan nilai mulai kerja nasabah ['start_career']  
- Melakukan perubahan untuk nilai mulai nasabah bekerja yang kurang dari 16 Tahun dan dimanipulasi menjadi 16 tahun, dengan alasan sebagian besar nasabah memiliki pendidikan akhir 'secondary education'. yang artinya lulus diusia 16 tahun dengan asumsi nasabah langsung bekerja setelah lulus.
</div>

In [24]:
# Atasi nilai yang bermasalah, jika ada
data['days_employed'] = data['days_employed'].abs()

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

Melakukan perubahan nilai negatif menjadi nilai positif


In [25]:
data['experience'] = round(data['days_employed'] / 365, 0)

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

Melakukan convert hari menjadi tahun.

In [26]:
data['start_career'] = data['dob_years'] - data['experience']

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

Melakukan perbandingan antara usia nasabah ['dob_years'] dengan ['experience']

In [27]:
data.loc[data['start_career'] <= 16, 'days_employed'] = 2190.0

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

Melakukan manipulasi untuk nilai mulai nasabah bekerja yang kurang dari 16 Tahun.

In [28]:
# Periksa hasilnya - pastikan telah diperbaiki
print(data['days_employed'].head(10))

0    8437.673028
1    4024.803754
2    5623.422610
3    4124.747207
4    2190.000000
5     926.185831
6    2879.202052
7     152.779569
8    2190.000000
9    2188.756445
Name: days_employed, dtype: float64


<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

Data jauh lebih baik dibanding sebelumnya.

In [29]:
# Periksa `dob_years` untuk nilai yang mencurigakan dan hitung persentasenya
print(data['dob_years'].unique())

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]


In [30]:
print(len(data.loc[data['dob_years'] == 0]) / len(data))

0.004692218350754936


<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>
 
Terdapat nilai yang berisi 0 pada data dengan persentase 0.4% dari data. Solusinya data tersebut akan dimanipulasi dengan nilai median.

In [31]:
# Atasi masalah pada kolom `dob_years`, jika terdapat masalah
data['dob_years'].median()

42.0

In [32]:
data.loc[data['dob_years'] == 0, 'dob_years'] = 42

In [33]:
# Periksa hasilnya - pastikan telah diperbaiki
print(data['dob_years'].unique())

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51 59 29 60 55 58 71
 22 73 66 69 19 72 70 74 75]


In [34]:
# Mari kita lihat nilai untuk kolom
data['family_status'].value_counts()

family_status
married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: count, dtype: int64

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

Tidak terdapat masalah pada kolom family_status

In [35]:
# Mari kita melihat nilainya di kolom
data['gender'].value_counts()


gender
F      14236
M       7288
XNA        1
Name: count, dtype: int64

In [36]:
data.loc[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,experience,start_career
10701,0,2358.600502,24,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate,6.0,18.0


<div class="alert alert-block alert-info">
<b>Noted.</b> <a class="tocSkip"></a>
    
Terdapat 1 value yang memiliki nilai string 'XNA', Solusinya akan diubah menjadi nilai rata-rata pada kolom tersebut.

In [37]:
# Atasi nilai-nilai yang bermasalah
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'

In [38]:
# Periksa hasilnya - pastikan telah diperbaiki
data['gender'].value_counts()

gender
F    14237
M     7288
Name: count, dtype: int64

In [39]:
# Mari kita lihat nilai dalam kolom
data['income_type'].value_counts()

income_type
employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: count, dtype: int64

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

Tidak terdapat masalah pada kolom income type

In [41]:
# Memeriksa duplikat
data.duplicated().sum()

72

In [42]:
data[data.duplicated()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,experience,start_career
2849,0,NaN,41,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family,NaN,NaN
3290,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding,NaN,NaN
4182,1,NaN,34,bachelor's degree,0,civil partnership,1,F,employee,0,NaN,wedding ceremony,NaN,NaN
4851,0,NaN,60,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony,NaN,NaN
5557,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding,NaN,NaN
6312,0,NaN,30,secondary education,1,married,0,M,employee,0,NaN,building a real estate,NaN,NaN
7808,0,NaN,57,secondary education,1,civil partnership,1,F,retiree,0,NaN,having a wedding,NaN,NaN
7921,0,NaN,64,bachelor's degree,0,civil partnership,1,F,retiree,0,NaN,having a wedding,NaN,NaN
7938,0,NaN,71,secondary education,1,civil partnership,1,F,retiree,0,NaN,having a wedding,NaN,NaN
8583,0,NaN,58,bachelor's degree,0,unmarried,4,F,retiree,0,NaN,supplementary education,NaN,NaN


In [43]:
# Atasi duplikat,
data = data.drop_duplicates().reset_index(drop=True)

In [44]:
# Memeriksa duplikat
data.duplicated().sum()

0

In [45]:
# Periksa ukuran dataset yang sekarang Anda miliki setelah manipulasi pertama yang Anda lakukan
data.shape

(21453, 14)

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

Terdapat pengurangan jumlah baris data sebanyak 72 baris dengan alasan terdapat data yang terduplikat / ganda dan terdapat penambahan 2 kolom kategori data yaitu :
- *`experience`* - Lama nasabah bekerja (tahun)
- *`start_career`* - Nasabah mulai bekerja

# Bekerja dengan nilai yang hilang

In [46]:
# Temukan dictionary
print(data['dob_years'].unique())

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51 59 29 60 55 58 71
 22 73 66 69 19 72 70 74 75]


<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

Tidak terdapat masalah pada kolom dob_years

### Memperbaiki nilai yang hilang di `total_income`

In [47]:
# Mari menulis fungsi untuk menghitung kategori usia
def client_age(age):
    if age >= 64:
        return 'Old Adults'
    if 25 <= age <= 63:
        return 'Middle-aged Adults'
    if age <= 24:
        return 'Young Adults'

In [48]:
# Lakukan pengujian apakah fungsi bekerja atau tidak
print(client_age(16))
print(client_age(45))
print(client_age(64))

Young Adults
Middle-aged Adults
Old Adults


In [49]:
# Membuat kolom baru berdasarkan fungsi
data['dob_years_category'] = data['dob_years'].apply(client_age)

In [50]:
# Memeriksa bagaimana nilai di dalam kolom baru
data['dob_years_category'].value_counts()

dob_years_category
Middle-aged Adults    19423
Old Adults             1155
Young Adults            875
Name: count, dtype: int64

In [51]:
# Membuat tabel tanpa nilai yang hilang dan menampilkan beberapa barisnya untuk memastikan semuanya berjalan dengan baik
data_nan_filtered = data.loc[~(data['days_employed'].isna())]

In [52]:
# Perhatikan nilai rata-rata untuk pendapatan berdasarkan faktor yang telah Anda identifikasi
data_nan_filtered['total_income'].mean()

26787.568354658673

In [53]:
# Perhatikan nilai median untuk pendapatan berdasarkan faktor yang telah Anda identifikasi
data_nan_filtered['total_income'].median()

23202.87

In [54]:
data_nan_filtered['total_income'].sort_values(ascending=False).head(20)

12390    362496.645
19547    352136.354
9159     276204.162
20741    274402.943
17136    273809.483
17459    255618.158
18320    248184.463
18305    228469.514
15233    216039.297
11052    205804.960
2224     204579.617
7441     198426.400
19753    195686.797
15176    187593.573
13066    180613.801
9856     176552.869
9989     175672.758
18714    174817.312
19282    174660.414
1590     174259.249
Name: total_income, dtype: float64

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>
    
Untuk menentukan nilai yang hilang pada value pendapatan lebih baik menggunakan median pendapatannya.
    

In [55]:
#  Tulis fungsi yang akan kita gunakan untuk mengisi nilai yang hilang
def fill_value(data, column_filter, column_target, rules):
    for rule in rules:
        value = data.loc[(data[column_filter] == rule) & ~(data['days_employed'].isna()), column_target].median()
        data.loc[(data[column_filter] == rule) & (data[column_target].isna()), column_target] = value
    return data

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>
    
Membuat fungsi nilai median berdasarkan kategori

In [56]:
# Memeriksa bagaimana nilai di dalam kolom baru
data['total_income_missing'] = data['total_income']

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>
    
Membuat kolom baru untuk memeriksa hasil fungsi

In [57]:
rules_age = data['dob_years_category'].unique()

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>
    
Kategori berdasarkan usia

In [58]:
# Terapkan fungsi ke setiap baris
data = fill_value(data, 'dob_years_category', 'total_income_missing', rules_age)

In [59]:
# Periksa apakah kita mendapatkan kesalahan
print(data['total_income_missing'].isna().sum())

0


In [60]:
print(data['total_income_missing'].mean())

26437.169737892134


In [61]:
print(data['total_income_missing'])

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21448    35966.698
21449    24959.969
21450    14347.610
21451    39054.888
21452    13127.587
Name: total_income_missing, Length: 21453, dtype: float64


In [62]:
# Memeriksa jumlah entri di kolom
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21453 non-null  int64  
 1   days_employed         19351 non-null  float64
 2   dob_years             21453 non-null  int64  
 3   education             21453 non-null  object 
 4   education_id          21453 non-null  int64  
 5   family_status         21453 non-null  object 
 6   family_status_id      21453 non-null  int64  
 7   gender                21453 non-null  object 
 8   income_type           21453 non-null  object 
 9   debt                  21453 non-null  int64  
 10  total_income          19351 non-null  float64
 11  purpose               21453 non-null  object 
 12  experience            19351 non-null  float64
 13  start_career          19351 non-null  float64
 14  dob_years_category    21453 non-null  object 
 15  total_income_missin

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>
    
Data terlihat lebih baik dan tidak terdapat kendala

###  Memperbaiki nilai di `days_employed`

In [63]:
# Distribusi median dari `days_employed` berdasarkan parameter yang Anda identifikasi
for column in ['dob_years_category', 'gender']:
    print(data.groupby(column)['days_employed'].median())
    print()

dob_years_category
Middle-aged Adults    2180.801412
Old Adults            2190.000000
Young Adults           744.542130
Name: days_employed, dtype: float64

gender
F    2190.000000
M    1681.233351
Name: days_employed, dtype: float64



In [64]:
# Distribusi rata-rata dari `days_employed` berdasarkan parameter yang Anda identifikasi
for column in ['dob_years_category', 'gender']:
    print(data.groupby(column)['days_employed'].mean())
    print()

dob_years_category
Middle-aged Adults    2334.934694
Old Adults            2480.543620
Young Adults           855.827246
Name: days_employed, dtype: float64

gender
F    2391.380975
M    2074.283408
Name: days_employed, dtype: float64



<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>
    
Nilai Nan akan diubah menjadi value median berdasarkan kategori usia dengan alasan data median lebih masuk akal.

In [65]:
# Mari tulis fungsi yang menghitung rata-rata atau median (tergantung keputusan Anda) berdasarkan parameter yang Anda identifikasi
def clean_data(data, column_filter, column_selected, rules):
    for rule in rules:
        median = data.loc[(data[column_filter] == rule) & ~(data[column_selected].isna()), column_selected].median()
        data.loc[(data[column_filter] == rule) & (data[column_selected].isna()), column_selected] = median
    return data

In [66]:
# Periksa bahwa fungsi bekerja
clean_data(data, 'dob_years_category', 'total_income', rules_age)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,experience,start_career,dob_years_category,total_income_missing
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.0,19.0,Middle-aged Adults,40620.102
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.0,25.0,Middle-aged Adults,17932.802
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.0,18.0,Middle-aged Adults,23341.752
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.0,21.0,Middle-aged Adults,42820.568
4,0,2190.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,932.0,-879.0,Middle-aged Adults,25378.572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,12.0,31.0,Middle-aged Adults,35966.698
21449,0,2190.000000,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,942.0,-875.0,Old Adults,24959.969
21450,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,6.0,32.0,Middle-aged Adults,14347.610
21451,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,9.0,29.0,Middle-aged Adults,39054.888


In [67]:
# Terapkan fungsi ke total_income
data = clean_data(data, 'dob_years_category', 'total_income', rules_age)

In [68]:
# Periksa bahwa fungsi bekerja
data.isna().sum()

children                   0
days_employed           2102
dob_years                  0
education                  0
education_id               0
family_status              0
family_status_id           0
gender                     0
income_type                0
debt                       0
total_income               0
purpose                    0
experience              2102
start_career            2102
dob_years_category         0
total_income_missing       0
dtype: int64

In [69]:
# Mengganti nilai yang hilang
data = clean_data(data, 'dob_years_category', 'days_employed', rules_age)

In [70]:
# Periksa entri di semua kolom - pastikan kita memperbaiki semua nilai yang hilang
data.isna().sum()

children                   0
days_employed              0
dob_years                  0
education                  0
education_id               0
family_status              0
family_status_id           0
gender                     0
income_type                0
debt                       0
total_income               0
purpose                    0
experience              2102
start_career            2102
dob_years_category         0
total_income_missing       0
dtype: int64

In [71]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,experience,start_career,dob_years_category,total_income_missing
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.0,19.0,Middle-aged Adults,40620.102
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.0,25.0,Middle-aged Adults,17932.802
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.0,18.0,Middle-aged Adults,23341.752
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.0,21.0,Middle-aged Adults,42820.568
4,0,2190.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,932.0,-879.0,Middle-aged Adults,25378.572


In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21453 non-null  int64  
 1   days_employed         21453 non-null  float64
 2   dob_years             21453 non-null  int64  
 3   education             21453 non-null  object 
 4   education_id          21453 non-null  int64  
 5   family_status         21453 non-null  object 
 6   family_status_id      21453 non-null  int64  
 7   gender                21453 non-null  object 
 8   income_type           21453 non-null  object 
 9   debt                  21453 non-null  int64  
 10  total_income          21453 non-null  float64
 11  purpose               21453 non-null  object 
 12  experience            19351 non-null  float64
 13  start_career          19351 non-null  float64
 14  dob_years_category    21453 non-null  object 
 15  total_income_missin

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>
    
Proses manipulasi data Nan telah selesai dilakukan dan tidak terdapat kendala

## Pengkategorian Data

In [73]:
# Tampilkan nilai data yang Anda pilih untuk pengkategorian data
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,experience,start_career,dob_years_category,total_income_missing
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.0,19.0,Middle-aged Adults,40620.102
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.0,25.0,Middle-aged Adults,17932.802
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.0,18.0,Middle-aged Adults,23341.752
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.0,21.0,Middle-aged Adults,42820.568
4,0,2190.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,932.0,-879.0,Middle-aged Adults,25378.572


In [74]:
target_category_column = ['days_employed', 'total_income', 'purpose', 'debt']

In [75]:
# Periksa nilai unik
for i in target_category_column:
    print(i)
    print(data[i].unique())
    print()

days_employed
[8437.67302776 4024.80375385 5623.42261023 ... 2113.3468877  3112.4817052
 1984.50758853]

total_income
[40620.102 17932.802 23341.752 ... 14347.61  39054.888 13127.587]

purpose
['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'
 'second-hand car purchase' 'getting an education' 'car'
 'wedding ceremony' 'to get a supplementary education'
 'purchase of my own house' 'real estate transactions'
 'getting higher education' 'to own a car' 'purchase of a car'
 'profile education' 'university education'
 'buying property f

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>
    
Terdapat 4 kolom yang perlu dilakukan pengkategorian data

In [76]:
# Mari kita menulis fungsi untuk mengkategorikan data berdasarkan topik umum
def debt_info(value):
    if value == 1:
        return 'payment failed'
    if value == 0:
        return 'payment reached'
    else:
        return 'not defined'

In [77]:
# Buat kolom dengan kategori dan hitung nilainya
data['debt_meaning'] = data['debt'].apply(debt_info)

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>
    
Melakukan pengkategorian data debt yang dapat diperlukan untuk proses hipotesis

In [78]:
# Lihat melalui semua data numerik di kolom yang Anda pilih untuk pengkategorian
column_num_selected = ['days_employed','dob_years','total_income']
data['debt_meaning'].value_counts()

debt_meaning
payment reached    19712
payment failed      1741
Name: count, dtype: int64

In [79]:
# Mendapatkan kesimpulan statistik untuk kolomnya
data[column_num_selected].describe()

,days_employed,dob_years,total_income
count,21453.000000,21453.000000,21453.000000
mean,2266.842391,43.469025,26437.169738
std,1903.780164,12.214162,15687.907748
min,24.141633,19.000000,3306.762000
25%,1012.303236,33.000000,17219.352000
50%,2180.801412,42.000000,23590.955500
75%,2464.055714,53.000000,31331.009000
max,16119.687737,75.000000,362496.645000


In [80]:
# Membuat fungsi untuk pengkategorian menjadi kelompok numerik yang berbeda berdasarkan rentang
def total_income_id (value):
    if value <= 52000:
        return 'Low income'
    if 52001 <= value <= 156000:
        return 'Middle income'
    if value >= 156000:
        return 'Upper income'

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>
    
Melakukan pengkategorian data penghasilan yang mungkin diperlukan untuk proses hipotesis

In [81]:
data['total_income_category'] = data['total_income'].apply(total_income_id)

In [82]:
data['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [83]:
def purpose_meaning(value):
    for word in value.split():
        if 'house' in word:
            return 'house'
        if 'real estate' in word:
            return 'house'
        if 'housing' in word:
            return 'house'
        if 'wedding' in word:
            return 'wedding'
        if 'car' in word:
            return 'car'
        if 'cars' in word:
            return 'car'
        if 'education' in word:
            return 'education'
        if 'university' in word:
            return 'education'
        if 'educated' in word:
            return 'education'
        if 'property' in word:
            return 'property'

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>
    
Melakukan pengkategorian data tujuan yang mungkin diperlukan untuk proses hipotesis

In [84]:
data['purpose_category'] = data['purpose'].apply(purpose_meaning)

In [85]:
data['purpose_category'].value_counts()

purpose_category
car          4306
education    4013
house        3809
property     2538
wedding      2323
Name: count, dtype: int64

In [86]:
# Membuat kolom dengan kategori
category_column = ['gender', 'children', 'education', 'family_status', 'income_type', 'dob_years_category', 'total_income_category', 'purpose_category', 'debt_meaning']

In [87]:
# Menghitung setiap nilai kategori untuk melihat pendistribusian
for i in category_column:
    print(i)
    print(data[i].value_counts())
    print()

gender
gender
F    14174
M     7279
Name: count, dtype: int64

children
children
0    14090
1     4855
2     2128
3      330
4       41
5        9
Name: count, dtype: int64

education
education
secondary education    15171
bachelor's degree       5250
some college             744
primary education        282
graduate degree            6
Name: count, dtype: int64

family_status
family_status
married              12339
civil partnership     4150
unmarried             2810
divorced              1195
widow / widower        959
Name: count, dtype: int64

income_type
income_type
employee                       11083
business                        5078
retiree                         3829
civil servant                   1457
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: count, dtype: int64

dob_years_category
dob_years_category
Middle-aged Adults    19423
Old Adults             1155
You

<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>
    
Data sudah terkategori dengan baik.

## Memeriksa Hipotesis


**Apakah terdapat korelasi antara memiliki anak dengan membayar kembali tepat waktu?**

In [88]:
# Periksa data anak dan membayar kembali dengan tepat waktu
data_pivot_children = data.pivot_table(index='children', columns='debt_meaning', values='debt', aggfunc='count')
print(data_pivot_children)

debt_meaning  payment failed  payment reached
children                                     
0                     1063.0          13027.0
1                      445.0           4410.0
2                      202.0           1926.0
3                       27.0            303.0
4                        4.0             37.0
5                        NaN              9.0


In [89]:
# Menghitung tarif otomatis berdasarkan jumlah anak
print(data_pivot_children['payment failed'] / (data_pivot_children['payment failed'] + data_pivot_children['payment reached']) *100)

children
0    7.544358
1    9.165808
2    9.492481
3    8.181818
4    9.756098
5         NaN
dtype: float64


<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>

persentase tertinggi gagal bayar berdasarkan kategori anak terdapat pada jumlah anak 4 yaitu 9,7% dari total nasabah yang memiliki anak 4 dan terendah jumlah anak 5 yaitu 0%


**Apakah terdapat korelasi antara status keluarga dengan membayar kembali tepat waktu?**

In [90]:
# Periksa data status keluarga dan membayar kembali dengan tepat waktu
data_pivot_family = data.pivot_table(index=['family_status_id', 'family_status'], columns='debt_meaning', values='debt', aggfunc='count')
print(data_pivot_family)

debt_meaning                        payment failed  payment reached
family_status_id family_status                                     
0                married                       931            11408
1                civil partnership             388             3762
2                widow / widower                63              896
3                divorced                       85             1110
4                unmarried                     274             2536


In [91]:
# Menghitung tarif otomatis berdasarkan status keluarga
print(data_pivot_family['payment failed'] / (data_pivot_family['payment failed'] + data_pivot_family['payment reached']) *100)

family_status_id  family_status    
0                 married              7.545182
1                 civil partnership    9.349398
2                 widow / widower      6.569343
3                 divorced             7.112971
4                 unmarried            9.750890
dtype: float64


**Kesimpulan**
<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>
    
persentase tertinggi gagal dalam kategori keluarga yaitu terdapat pada status keluarga yang belum menikah yaitu 9,7% dari total keseluruhan basabah yang belum menikah dan terendah dengan status janda yaitu 6,5%

**Apakah terdapat korelasi antara tingkat pendapatan dengan membayar kembali tepat waktu?**

In [92]:
# Periksa data tingkat pendapatan dan membayar kembali dengan tepat waktu
data_pivot_income = data.pivot_table(index='total_income_category', columns='debt_meaning', values='debt', aggfunc='count')
print(data_pivot_income)

debt_meaning           payment failed  payment reached
total_income_category                                 
Low income                       1659            18631
Middle income                      80             1057
Upper income                        2               24


In [93]:
# Menghitung tarif otomatis berdasarkan tingkat pendapatan
print(data_pivot_income['payment failed'] / (data_pivot_income['payment failed'] + data_pivot_income['payment reached']) *100)

total_income_category
Low income       8.176442
Middle income    7.036060
Upper income     7.692308
dtype: float64


**Kesimpulan**

<div class="alert alert-block alert-info">
<b>Student answer.</b> <a class="tocSkip"></a>
    
persentase tertinggi gagal bayar pada kategori pendapatan terdapat pada kategori pendapatan rendah yaitu 8,1% dari total keseluruhan nasabah yang memiliki pendapatan rendah dan persentase terendah dengan pendapatan menengah yaitu 7%

**Bagaimana tujuan kredit memengaruhi tarif otomatis?**

In [94]:
data_pivot_purpose = data.pivot_table(index='purpose_category', columns='debt_meaning', values='debt', aggfunc='count')
print(data_pivot_purpose)

debt_meaning      payment failed  payment reached
purpose_category                                 
car                          403             3903
education                    370             3643
house                        256             3553
property                     190             2348
wedding                      186             2137


In [95]:
print(data_pivot_purpose['payment failed'] / (data_pivot_purpose['payment failed'] + data_pivot_purpose['payment reached']) *100)

purpose_category
car          9.359034
education    9.220035
house        6.720924
property     7.486210
wedding      8.006888
dtype: float64


**Kesimpulan**
<div class="alert alert-block alert-info">
<b>Note.</b> <a class="tocSkip"></a>
    
persentase tertinggi gagal bayar pada kategori tujuan nasabah terdapat pada tujuan membeli mobil yaitu 9,3% dari total keseluruhan nasabah yang memiliki tujuan pembelian mobil dan terendah tujuan membeli rumah yaitu 6,7%

# Kesimpulan Umum 
<div class="alert alert-block alert-info">
<b> Note.</b> <a class="tocSkip"></a>
    
Beberapa metode sudah dilakukan untuk melakukan investigasi data yang hilang namun tidak ditemukan pola terhadap data yang hilang. berhubung data valid lebih besar yaitu 90% maka data yang hilang tersebut akan diisi dengan nilai yang lebih masuk akan yaitu dengan median data tersebut.
    <br/>
    Pertama-tama data yang tersedia dilakukan investigasi terlebih dahulu berupa : <br/>
    - pengecekan nilai hilang,<br/>
    - Pengecekan isi data, <br/><br/>
  berdasarkan hasil investigasi beberapa temukan masalah data ditemukan yaitu : <br/>
1. Terdapat isi data yang hilang yaitu berupa penghasilan dan lama nasabah bekerja dengan persentase 10% dari data total <br/>
   Adapun beberapa penanggulangan pertama yaitu <br/>
   - melakukan pengecekan terhadap nilai yang hilang apakah terdapat pola pada nilai yang hilang, namun hasil pengecekan tidak terdapat pola pada nilai yang hilang tersebut.<br/>
    - berhubung data valid lebih besar yaitu 90% maka data yang hilang tersebut akan diisi dengan nilai yang lebih masuk akal yaitu dengan median dari data tersebut berasarkan kategori umur untuk kedua data yang hilang. <p>
2. Terdapat ketidakselarasan dalam penulisan pendidikan, sehingga solusi yang dilakukan yaitu merubah isi tulisan sehingga terdapat keselarasan dan memudahkan proses analisis selanjutnya.<p>
3. Terdapat nilai negatif pada informasi durasi lamanya nasabah telah bekerja dan terdapat nilai yang telah bekerja lebih dari 36500 Hari atau 100 tahun. Solusi terhadap kendala tersebut yaitu mengubah nilai negatif menjadi positif dan melakukan  perbandingan antara lama bekerja 'days_employed' dan usia nasabah 'dob_years' dengan metode:<br/>
    - penyelarasan perhitungan menjadi tahun agar lebih mudah dalam investigasi dengan membuat column baru.<br/>
    - membandingkan lama bekerja dan usia pelanggan dengan argumen. Awal nasabah memulai karir yaitu 16 tahun. dengan alasan sebagian besar nasabah memiliki pendidikan akhir 'secondary education' yang artinya lulus diusia 16 tahun dan asumsi nasabah langsung bekerja setelah lulus. dan untuk value nasabah yang memulai karir kurang dari 16 tahun. lama bekerja akan diubah menjadi median dari lama bekerja.<p>
    setelah investigasi atau manipulasi data telah selesai dilakukan, data dapat dikaji dan berikut kesimpulan yang didapat : <p>
    1. persentase tertinggi gagal bayar berdasarkan kategori anak terdapat pada jumlah anak 4 yaitu 9,7% dari total nasabah yang memiliki anak 4 dan terendah jumlah anak 5 yaitu 0%<p>
    2. persentase tertinggi gagal dalam kategori keluarga yaitu terdapat pada status keluarga yang belum menikah yaitu 9,7% dari total keseluruhan basabah yang belum menikah dan terendah dengan status janda yaitu 6,5% <p>
    3. persentase tertinggi gagal bayar pada kategori pendapatan terdapat pada kategori pendapatan rendah yaitu 8,1% dari total keseluruhan nasabah yang memiliki pendapatan rendah dan persentase terendah dengan pendapatan menengah yaitu 7%.<p>
    4. persentase tertinggi gagal bayar pada kategori tujuan nasabah terdapat pada tujuan membeli mobil yaitu 9,3% dari total keseluruhan nasabah yang memiliki tujuan pembelian mobil dan terendah tujuan membeli rumah yaitu 6,7%
  
</div>
